# Импорт необходимых объектов

In [1]:
from warnings import filterwarnings
filterwarnings('ignore')

# Dataset
from sklearn.datasets import load_digits

# Split
from sklearn.model_selection import train_test_split

# Models
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier

# Dimension reduction
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE

# Metrics
from sklearn.metrics import classification_report

# Time
from time import time

# Параметры

In [2]:
random_state = 42
test_size = 0.20

# Исходные данные

In [3]:
digits = load_digits()
X = digits.data
y = digits.target

# 1. Обучение моделей SVM, logisitic regression, decision tree на исходных данных

In [4]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size,  
                                                    random_state=random_state)

In [5]:
models = {
    'SVM': SVC(random_state=random_state),
    'Logistic regression': LogisticRegression(random_state=random_state),
    'Decision tree': DecisionTreeClassifier(random_state=random_state)
}

# Train, predict and show results
for model_name, model_object in models.items():
    model_object.fit(X_train, y_train)
    y_predict = model_object.predict(X_test)
    print(model_name, '\n', classification_report(y_test, y_predict))

SVM 
               precision    recall  f1-score   support

           0       1.00      1.00      1.00        33
           1       1.00      1.00      1.00        28
           2       1.00      1.00      1.00        33
           3       1.00      1.00      1.00        34
           4       1.00      1.00      1.00        46
           5       0.98      0.98      0.98        47
           6       0.97      1.00      0.99        35
           7       0.97      0.97      0.97        34
           8       1.00      0.97      0.98        30
           9       0.95      0.95      0.95        40

    accuracy                           0.99       360
   macro avg       0.99      0.99      0.99       360
weighted avg       0.99      0.99      0.99       360

Logistic regression 
               precision    recall  f1-score   support

           0       1.00      1.00      1.00        33
           1       0.97      1.00      0.98        28
           2       0.97      1.00      0.99       

# 2. Использование алгоритмов PCA и t-SNE для снижения размерности

In [6]:
dimension_reduction_tools = {
    'PCA': PCA(n_components=0.90, random_state=random_state),
    't-SNE': TSNE(random_state=random_state)
}

# Get X processed with time
X_processed = {}
processing_time = {}
for tool_name in dimension_reduction_tools.keys():
    start_time = time()
    
    # The same actions demand different attributes
    try:
        dimension_reduction_tools[tool_name].fit(X)
        X_processed[tool_name] = dimension_reduction_tools[
            tool_name].transform(X)
    except AttributeError:
        X_processed[tool_name] = dimension_reduction_tools[
            tool_name].fit_transform(X)
    
    processing_time[tool_name] = time() - start_time

### Какое количество компонент необходимо, чтобы описать 90% дисперсии в PCA?

In [7]:
print(dimension_reduction_tools['PCA'].n_components_)

21


# 3. Обучение тех же моделей на сниженном признаковом пространстве

In [8]:
for tool_name, tool in dimension_reduction_tools.items():
    print(tool_name)
    
    X_train, X_test, y_train, y_test = train_test_split(
        X_processed[tool_name], y,
        test_size=test_size, random_state=random_state)
    
    for model_name, model_object in models.items():
        model_object.fit(X_train, y_train)
        y_predict = model_object.predict(X_test)
        print(model_name, '\n', classification_report(y_test, y_predict))

PCA
SVM 
               precision    recall  f1-score   support

           0       1.00      1.00      1.00        33
           1       1.00      1.00      1.00        28
           2       1.00      1.00      1.00        33
           3       1.00      1.00      1.00        34
           4       1.00      1.00      1.00        46
           5       0.98      0.98      0.98        47
           6       0.97      1.00      0.99        35
           7       0.97      0.97      0.97        34
           8       1.00      0.97      0.98        30
           9       0.95      0.95      0.95        40

    accuracy                           0.99       360
   macro avg       0.99      0.99      0.99       360
weighted avg       0.99      0.99      0.99       360

Logistic regression 
               precision    recall  f1-score   support

           0       0.94      0.97      0.96        33
           1       0.96      0.96      0.96        28
           2       0.91      0.97      0.94   

### Какой алгоритм снижения дает лучшие метрики точности моделей классификации?

Как можно видеть, для разных моделей классификации результаты отличаются.
В целом, алгоритм t-SNE выдал accuracy, больший 0.9 для всех моделей классификации, хотя и для SVM и для логистической регрессии он показал результаты, которые хуже аналогичных результатов, показанных алгоритмом PCA.

### Есть ли улучшение относительно работы моделей на исходном признаковом пространстве?

Однозначно, улучшения наблюдаются у алгоритма дерева решений (повышение accuracy). А вот результаты алгоритмов SVM и логистической регрессии после уменьшения размерности стали хуже.

# 4. Оценка количества времени, которое требуется для препроцессинга PCA и t-SNE

In [9]:
for tool_name, tool_processing_time in processing_time.items():
    print(tool_name, tool_processing_time, 'с')

PCA 0.028592586517333984 с
t-SNE 27.184375047683716 с
